In [1]:
import pandas as pd

In [2]:
#For using datasets in our drive
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#Get to data - 
%cd 'drive/My Drive/Enlight'

/content/drive/My Drive/Enlight


In [5]:
df = pd.read_csv("cleaned_news.csv")

In [6]:
df.head()

,text,fake_news
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


#Naive Multinomial Bayes with Sklearn

In [7]:
#Split data into independent and dependent variables
DV = "fake_news" #the dependent variable, text is independent here
X = df.drop([DV], axis = 1) #drop from our X array because this it the text data to be trained
Y = df[DV]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#Train 75% of the data, 25% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25)

In [10]:
#Convert words into bag of words model with sklearns CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vect = CountVectorizer(max_features = 5000) #limit to 5000 unique words, but room to mess around is here

X_train_counts = count_vect.fit_transform(X_train['text'])
#print(count_vect.vocabulary_) #here is our bag of words

X_test = count_vect.transform(X_test['text']) # note, this is not fit to the model

In [12]:
print(count_vect.vocabulary_)

{'best': 573, 'known': 2543, 'as': 410, 'new': 3033, 'york': 4983, 'hedge': 2124, 'fund': 1916, 'industry': 2312, 'executive': 1648, 'anthony': 336, 'scaramucci': 3959, 'president': 3439, 'donald': 1422, 'trump': 4647, 'incoming': 2287, 'communications': 966, 'director': 1360, 'has': 2091, 'in': 2276, 'film': 1797, 'company': 971, 'manhattan': 2769, 'and': 312, 'business': 706, 'accused': 145, 'by': 714, 'regulators': 3719, 'of': 3103, 'making': 2756, 'false': 1729, 'claims': 885, '2015': 40, 'financial': 1802, 'show': 4109, 'overall': 3197, 'assets': 429, 'range': 3620, 'approximately': 380, 'million': 2875, 'to': 4565, '85': 105, 'the': 4502, 'forms': 1869, 'he': 2105, 'also': 286, 'such': 4352, 'personal': 3311, 'loans': 2687, 'between': 576, '25': 53, 'income': 2286, 'since': 4134, 'start': 4263, '2016': 41, 'more': 2932, 'than': 4498, '10': 2, 'is': 2407, 'mostly': 2939, 'from': 1905, 'capital': 749, 'investment': 2388, 'that': 4501, 'founded': 1878, '2005': 30, 'now': 3074, 'proc

#Naive Bayes

In [16]:
from sklearn.naive_bayes import MultinomialNB

In [17]:
#fit the training dataset on the NB classifier
Naive = MultinomialNB() #create model
Naive.fit(X_train_counts, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
#Test Accuracy
from sklearn.metrics import accuracy_score

In [19]:
#predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)

In [20]:
#Use accuracy score fn to get accuracy. Very high accuracy b/c of assumption of independence
print("Accuracy Score:", accuracy_score(predictions_NB, Y_test)*100)

Accuracy Score: 94.37511142806204


#Onion Test

In [21]:
# link: https://entertainment.theonion.com/drake-fans-accuse-kenny-chesney-of-manipulating-billboa-1843484082
onion = ["""Calling the country singer’s place at the top of Top 200 completely illegitimate, fans of the 
            rapper–singer Drake took to social media Friday to accuse Kenny Chesney of manipulating Billboard’s 
            algorithm by putting effort into his album. “It’s just unfair that this guy could keep Drake from his
            rightful place on the charts by putting out quality music that he actually cares about,” said Aiden 
            Howard, 14, who echoed the sentiments of Drake fans worldwide in his assertion that the artist’s 
            mediocre B-sides deserved more acclaim and recognition. “He clearly gamed the streaming numbers when 
            he decided to put time and energy into his craft. It’s such horseshit that Billboard rewards that 
            behavior and punishes Drizzy for making a half-assed mixtape full of songs he’d already dropped on 
            SoundCloud. How the hell is ‘Toosie Slide’ going to compare to a song that the artist thought about 
            for more than 15 minutes?” At press time, Drake released a statement asking fans to ignore Kenny 
            Chesney and focus on the horseshit that he just released."""]

In [23]:
onion_vec = count_vect.transform(onion) #create bag of words

In [24]:
predict_onion = Naive.predict(onion_vec) #apply it to trained model

In [25]:
print(predict_onion) #1 = fake news

[1]


#New York Times' Test

In [26]:
# link: https://www.nytimes.com/2020/05/16/us/politics/linick-investigation-pompeo.html?action=click&module=Top%20Stories&pgtype=Homepage
nyt = ["""Two top congressional Democrats opened an investigation on Saturday into President Trump’s removal of 
          Steve A. Linick, who led the office of the inspector general at the State Department, citing a pattern 
          of “politically-motivated firing of inspectors general.” Mr. Trump told Speaker Nancy Pelosi late 
          Friday night that he was ousting Mr. Linick, who was named by President Barack Obama to the State 
          Department post, and replacing him with an ambassador with close ties to Vice President Mike Pence in 
          the latest purge of inspectors general whom Mr. Trump has deemed insufficiently loyal to his 
          administration. In letters to the White House, State Department, and Mr. Linick, Representative Eliot 
          L. Engel of New York, the chairman of the House Foreign Affairs Committee, and Senator Bob Menendez of 
          New Jersey, the top Democrat on the Senate Foreign Relations Committee, requested that the administration
          turn over records and information related to the firing of Mr. Linick as well as “records of all I.G. 
          investigations involving the Office of the Secretary that were open, pending, or incomplete at the 
          time of Mr. Linick’s firing.” Mr. Engel and Mr. Menendez said in their letters that they believe 
          Secretary of State Mike Pompeo recommended Mr. Linick’s ouster because he had opened an investigation 
          into Mr. Pompeo’s conduct. The lawmakers did not provide any more details, but a Democratic aide said 
          that Mr. Linick had been looking into whether Mr. Pompeo had misused a political appointee at the State 
          Department to perform personal tasks for himself and his wife. “Such an action, transparently designed to
          protect Secretary Pompeo from personal accountability, would undermine the foundation of our democratic 
          institutions and may be an illegal act of retaliation,” the lawmakers wrote. Under law, the administration
          must notify Congress 30 days before formally terminating an inspector general. Mr. Linick is expected to 
          leave his post then. Mr. Trump’s decision to remove Mr. Linick is the latest in a series of ousters aimed
          at inspectors general who the president and his allies believe are opposed to his agenda. In May, Mr. 
          Trump moved to oust Christi A. Grimm, the principal deputy inspector general for the Department of Health
          and Human Services, whose office had issued a report revealing the dire state of the nation’s response to
          the pathogen. He has also taken steps to remove two other inspectors general, for the intelligence
          community and for the Defense Department. Mr. Linick was spotlighted during the impeachment inquiry when 
          he requested an urgent meeting with congressional staff members to give them copies of documents related 
          to the State Department and Ukraine, signaling they could be relevant to the House investigation into 
          whether President Trump pressured Ukraine to investigate former Vice President Joseph R. Biden Jr. and 
          his son Hunter Biden. The documents — a record of contacts between Rudolph W. Giuliani, the president’s 
          personal lawyer, and Ukrainian prosecutors, as well as accounts of Ukrainian law enforcement proceedings 
          — turned out to be largely inconsequential."""]

In [27]:
nyt_vec = count_vect.transform(nyt) #create bag of words

In [30]:
predict_nyt = Naive.predict(nyt_vec) #apply to model

In [31]:
print(predict_nyt) #0 for false

[0]
